In [10]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

## Part b

In [11]:
epsilon = 0.2
mu = 0.1
delta_x = 0.4
delta_t = 0.1
N_x = 130
L = N_x*delta_x

In [12]:
u_0 = np.zeros(N_x)
for j in range(len(u_0)):
    u_0[j] = 0.5*(1-np.tanh((j-25)/5))

In [13]:
#Checking the stability condition:
(delta_t/delta_x)*(epsilon*np.sqrt(np.sum([u_0[j]**2 for j in range(N_x)]))+4*(mu/(delta_x**2)))

0.8647925385270318

The stability condition is fulfilled, because 0.865 < 1

In [55]:
def apply_boundary_conditions(arr, arrs_prev, isfirst=False): 
    """
    :param arr: array that boundary conditions are applied on
    :param arrs_prev: array before the current array and the array before that in case of isfirst=False
    :param isfirst: is arr u_1?
    :return: 
    """
    if isfirst:
        arrs_prev = arrs_prev[np.newaxis, :]
    arr[0] = 1
    arr[-1] = 0
    a = arrs_prev[1][-2] if not isfirst else arrs_prev[0][-2]
    b = arrs_prev[1][1] if not isfirst else arrs_prev[0][1]
    pref = 1 if not isfirst else 0.5
    arr[-2] = a - pref*(epsilon*delta_t/3*delta_x)*(arrs_prev[0][-1]+arrs_prev[0][-2]+arrs_prev[0][-3])*(arrs_prev[0][-1]-arrs_prev[0][-3]) - pref*(mu*delta_t/(delta_x**3))*(-arrs_prev[0][-1]+2*arrs_prev[0][-3]-arrs_prev[0][-4])
    arr[1] = b - pref*(epsilon*delta_t/3*delta_x)*(arrs_prev[0][2]+arrs_prev[0][1]+arrs_prev[0][0])*(arrs_prev[0][2]-arrs_prev[0][0]) - pref*(mu*delta_t/(delta_x**3))*(arrs_prev[0][3]+arrs_prev[0][0]-2*arrs_prev[0][2])
    arr_final = arr
    return arr_final

def Koorteweg_de_Vries(num_timesteps, u_start):
    u_1 = np.zeros(N_x)
    u_1 = apply_boundary_conditions(u_1, u_start, isfirst=True)
    for j in range(2, N_x-2):
        u_1[j] = u_start[j] - (epsilon / 6) * (delta_t / delta_x) * (u_start[j + 1] + u_start[j] + u_start[j - 1]) * (u_start[j + 1] + u_start[j - 1]) - mu * (delta_t / (delta_x ** 3)) * (u_start[j + 2] + 2 * u_start[j - 1] - 2 * u_start[j + 1] - u_start[j - 2])
    u = [u_start, u_1]
    for t in tqdm(range(num_timesteps-1)):
        u_prev = u[-2] #n-1
        u_current = u[-1] #n
        u_next = np.zeros(N_x)
        u_next = apply_boundary_conditions(u_next, [u_current, u_prev])
        if np.isnan(u_next).any() or np.isinf(u_next).any():
            print("NaN or Inf detected. Stopping simulation.")
            break
        for j in range(2,N_x-2):
            u_next[j] = u_prev[j] - ((epsilon*delta_t)/(3*delta_x))*(u_current[j+1]+u_current[j]+u_current[j-1])*(u_current[j+1]+u_current[j-1]) - mu*(delta_t / (delta_x**3))*(u_current[j+2]+2*u_current[j-1]-2*u_current[j+1]-u_current[j-2])
        u.append(u_next)
    return u

In [56]:
u_KdV = Koorteweg_de_Vries(2000, u_0)

  0%|          | 0/1999 [00:00<?, ?it/s]C:\Users\corin\AppData\Local\Temp\ipykernel_15532\3993694988.py:35: RuntimeWarning: overflow encountered in scalar multiply
  u_next[j] = u_prev[j] - ((epsilon*delta_t)/(3*delta_x))*(u_current[j+1]+u_current[j]+u_current[j-1])*(u_current[j+1]+u_current[j-1]) - mu*(delta_t / (delta_x**3))*(u_current[j+2]+2*u_current[j-1]-2*u_current[j+1]-u_current[j-2])
C:\Users\corin\AppData\Local\Temp\ipykernel_15532\3993694988.py:35: RuntimeWarning: invalid value encountered in scalar subtract
  u_next[j] = u_prev[j] - ((epsilon*delta_t)/(3*delta_x))*(u_current[j+1]+u_current[j]+u_current[j-1])*(u_current[j+1]+u_current[j-1]) - mu*(delta_t / (delta_x**3))*(u_current[j+2]+2*u_current[j-1]-2*u_current[j+1]-u_current[j-2])
 18%|█▊        | 350/1999 [00:00<00:00, 6870.89it/s]

NaN or Inf detected. Stopping simulation.


In [49]:
print(u_KdV[330])

[ 1.00000000e+00  3.00218486e-01  4.54836921e-01 -8.38368129e-02
  2.07199886e-01 -2.47713437e-01  1.10178965e-01 -2.89972368e-01
  9.77733167e-02 -2.75227048e-01  1.09303140e-01 -2.53997515e-01
  1.17778086e-01 -2.51975283e-01  1.14639876e-01 -2.49537514e-01
  1.09444757e-01 -2.32800232e-01  1.33042420e-01 -2.06055102e-01
  1.68465745e-01 -1.74337000e-01  1.73195558e-01 -1.79713352e-01
  1.43662708e-01 -2.40825984e-01  7.50914078e-02 -3.15139318e-01
 -3.20355843e-02 -4.21326695e-01 -2.07423435e-01 -6.34755461e-01
 -1.28822460e-01  1.16709207e+00 -1.83104296e+01 -5.08442252e+02
 -4.31646609e+03 -1.60717573e+04 -3.02432280e+04 -3.14452161e+04
 -1.83868869e+04 -5.88683000e+03 -1.07960578e+03 -1.54431909e+02
 -2.88964124e+01 -8.96065638e+00 -4.23063586e+00 -2.59732663e+00
 -1.82061214e+00 -1.40690022e+00 -1.11427554e+00 -9.50386430e-01
 -7.83502343e-01 -7.16964569e-01 -5.95293480e-01 -5.74642920e-01
 -4.78504979e-01 -4.79902137e-01 -3.95913855e-01 -4.17728738e-01
 -3.30933445e-01 -3.72436